<a href="https://colab.research.google.com/github/andrewyzhuang/mae271D/blob/Janet_Ann_Weird_Existing_Code/Testing_Implementation_LMPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Needed Libraries
import numpy as np
import cvxpy as cp
import pdb

In [49]:
# Function
# Initial feasible trajectory (for initialization of sample safe set) using brute-force or open-loop control
# Function to compute initial feasible trajectory using a naive controller
def initial_trajectory(A, B, Q, R, x0, xF, u_min, u_max, tol, max_steps):
    x_traj = [x0]
    u_traj = []
    x = x0.copy()
    for _ in range(max_steps):
        # Weak, inefficient control: slow damping of velocity
        u = np.clip(-0.2 * x[1] - 0.01 * x[0], u_min, u_max)
        x = A @ x + B @ u
        x_traj.append(x.copy())
        u_traj.append(np.array([u]))
        if np.linalg.norm(x - xF) < tol:
            break
    return np.array(x_traj), np.array(u_traj)

In [56]:
# Function
# Compute cost-to-go for each point in trajectory
def compute_cost_to_go(x_traj, u_traj, Q, R):
    costs = []
    for t in range(len(x_traj)):
      cost = 0.0 # Initialize cost as a float
      # Iterate over the future trajectory from the current point t
      for k in range(t, len(x_traj) - 1):
        x = x_traj[k]
        u = u_traj[k]
        # Ensure cost contributions are added as scalars
        cost += (x.T @ Q @ x).item() + (u.T @ R @ u).item()

      # Add terminal cost from the last state, ensuring it's a scalar
      if len(x_traj) > 0: # Check if trajectory is not empty
        cost += (x_traj[-1].T @ Q @ x_traj[-1]).item()

      costs.append(cost)
    return np.array(costs)

In [64]:
# Function
# LMPC optimization solver with sampled safe set and terminal cost
# Use in each iteration of LMPC
def solve_lmpc(A, B, Q, R, N, x0, Qj, x_min, x_max, u_min, u_max):
    n, m = B.shape
    x = cp.Variable((n, N+1))
    u = cp.Variable((m, N))
    cost = 0
    constraints = [x[:, 0] == x0]

    # Construct cost and constraints
    for k in range(N):
        cost += cp.quad_form(x[:, k], Q) + cp.quad_form(u[:, k], R)
        constraints += [
            x[:, k+1] == A @ x[:, k] + B @ u[:, k],
            x_min <= x[:, k], x[:, k] <= x_max,
            u_min <= u[:, k], u[:, k] <= u_max
        ]

    # Terminal state should still satisfy bounds
    constraints += [x_min <= x[:, N], x[:, N] <= x_max]


    # Add the terminal cost
    cost += Qj


    prob = cp.Problem(cp.Minimize(cost), constraints) # problem
    prob.solve(solver=cp.OSQP) # apply solver to problem

    return u[:, 0].value

In [58]:
# Problem setup

# Dynamics
A = np.array([[1, 1],
              [0, 1]])
B = np.array([[0],
              [1]])

# Cost Weights
Q = np.eye(2)
R = np.eye(1)

# Constraints
x_min = np.array([-4, -4])
x_max = np.array([4, 4])
u_min = np.array([-1])
u_max = np.array([1])

# Horizon
N = 4

# Inital/Final States
xS = np.array([-3.95, -0.05]) # inital state x0
xF = np.array([0.0, 0.0]) # final desired state [0,0]
tol = 1e-3
max_iters = 6
max_steps = 100

In [65]:
# Implement

# LMPC loop initalizations
safe_set = []
trajectories = []
print("Starting LMPC...")

# Initialize with a feasible trajectory for first iteration
x_traj, u_traj = initial_trajectory(A, B, Q, R, xS, xF, u_min, u_max, tol=1e-3, max_steps=100)
trajectories.append((x_traj, u_traj))
safe_set.extend(x_traj)


# inital Qj
cost_to_go = compute_cost_to_go(x_traj, u_traj, Q, R)
Qj_init = np.min(cost_to_go)


# Iterate LMPC loop
for j in range(1, max_iters):
    print(f"\n=== Iteration {j} ===")
    x = xS.copy()
    x_traj = [x.copy()]
    u_traj = []

    if j ==1:
        Qj = Qj_init

    for step in range(max_steps):
        u = solve_lmpc(A, B, Q, R, N, x, Qj, x_min, x_max, u_min, u_max)
        x = A @ x + B @ u
        x_traj.append(x.copy())
        u_traj.append(u)
        if np.linalg.norm(x - xF) < tol:
            break

    # Store trajectory
    x_traj = np.array(x_traj)
    u_traj = np.array(u_traj)
    trajectories.append((x_traj, u_traj))

    # compute Qj for use in next iteration
    Qj = np.min(compute_cost_to_go(x_traj, u_traj, Q, R))


    # Update safe set and cost
    safe_set.extend(x_traj)

    # Compute and print iteration cost
    iter_cost = sum(
    x.T @ Q @ x + u.T @ R @ u
    for x, u in zip(x_traj[:-1], u_traj)
)
    final_cost = x_traj[-1].T @ Q @ x_traj[-1]
    total_cost = iter_cost + final_cost
    print(f"Iteration {j} cost: {total_cost:.4f}")

    print(f"Sampled Safe Set size: {len(safe_set)} points")

    # Debug: print first few states and controls of the current trajectory
    print("Initial state:", x_traj[0])
    print("First few controls:", u_traj[:3].flatten())
    print("First few states after control:", x_traj[1:4])

    print(f"Last point in new trajectory: {x_traj[-1]}")
    print(f"Distance to final target xF: {np.linalg.norm(x_traj[-1] - xF):.4f}")


print("\nLMPC completed.")



Starting LMPC...

=== Iteration 1 ===
Iteration 1 cost: 49.9291
Sampled Safe Set size: 114 points
Initial state: [-3.95 -0.05]
First few controls: [ 0.99999973  0.46000032 -0.47199996]
First few states after control: [[-4.          0.94999973]
 [-3.05000027  1.41000005]
 [-1.64000022  0.9380001 ]]
Last point in new trajectory: [ 0.00074408 -0.0006309 ]
Distance to final target xF: 0.0010

=== Iteration 2 ===
Iteration 2 cost: 49.9291
Sampled Safe Set size: 127 points
Initial state: [-3.95 -0.05]
First few controls: [ 0.99999973  0.46000032 -0.47199996]
First few states after control: [[-4.          0.94999973]
 [-3.05000027  1.41000005]
 [-1.64000022  0.9380001 ]]
Last point in new trajectory: [ 0.00074408 -0.0006309 ]
Distance to final target xF: 0.0010

=== Iteration 3 ===
Iteration 3 cost: 49.9291
Sampled Safe Set size: 140 points
Initial state: [-3.95 -0.05]
First few controls: [ 0.99999973  0.46000032 -0.47199996]
First few states after control: [[-4.          0.94999973]
 [-3.050